In [ ]:
import pandas as pd
import numpy as np

I need to make changes to how are weights computed, I should do the share, the the inverse of that then I need to have a sum of the inverses, and compute the weights. This way we are giving more weights to the treated units in the presence of large number of untreated units.
I need to add a function to replicate the way the authors compute the clustered standard errors

In [ ]:
def aggregate_treatment_effects(data, treatment_column, estimand='overall', groupby_column=None, time_column=None):
    """
    Aggregate treatment effects into a specific estimand.
    
    Parameters:
        data (DataFrame): The data containing the treatment effects.
        treatment_column (str): The column name for treatment effects.
        estimand (str): The type of estimand to calculate ('overall', 'cohort', 'event').
        groupby_column (str): The column to group by for cohort-specific estimands.
        time_column (str): The time column for event-specific estimands.
    
    Returns:
        float or Series: The aggregated treatment effect estimand.
    """
    
    if estimand == 'overall':
        # Calculating the overall treatment effect
        weights = 1 / data[treatment_column].count()
        weighted_treatment_effect = np.sum(weights * data[treatment_column])
        
        return weighted_treatment_effect
    
    elif estimand == 'cohort':
        # Check if a groupby_column is provided
        if groupby_column is None:
            raise ValueError('Groupby column must be provided for cohort-specific estimands.')
        
        group_data = data.groupby(groupby_column)
        weights = 1 / group_data[treatment_column].count()
        
        # Calculating cohort-specific average treatment effects
        weighted_treatment_effect = group_data[treatment_column].apply(np.mean) / weights
        
        return weighted_treatment_effect
    
    elif estimand == 'event':
        # Check if a time_column is provided
        if time_column is None:
            raise ValueError('Time column must be provided for event-specific estimands.')
            
        group_data = data.groupby(time_column)
        weights = 1 / group_data[treatment_column].count()
        
        # Calculating event-specific average treatment effects (Event study)
        weighted_treatment_effect = group_data[treatment_column].apply(np.mean) / weights
        
        return weighted_treatment_effect
    
    else:
        raise ValueError('Invalid estimand type. Choose among "overall", "cohort", or "event".')

In [ ]:
# Example usage
data = pd.DataFrame({'id': range(1, 11),
                     'treatment_effects': np.random.normal(0, 1, 10),
                     'cohort': ['A', 'B', 'A', 'B', 'A', 'A', 'B', 'A', 'B', 'A'],
                     'time': [1, 2, 1, 2, 1, 2, 1, 2, 1, 2]})

print(aggregate_treatment_effects(data, 'treatment_effects', estimand='overall'))
print(aggregate_treatment_effects(data, 'treatment_effects', estimand='cohort', groupby_column='cohort'))
print(aggregate_treatment_effects(data, 'treatment_effects', estimand='event', time_column='time'))